In [64]:
import polars as pl
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import numpy as np
import logging
import random

seed = 42
np.random.seed(seed)
random.seed(seed)
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.random.set_seed(seed)
print(tf.__version__)

2.16.1


In [65]:
history = pl.read_parquet('/Users/lorecampa/Desktop/Projects/RecSysChallenge2024/dataset/ebnerd_small/train/history.parquet')
articles = pl.read_parquet('/Users/lorecampa/Desktop/Projects/RecSysChallenge2024/dataset/ebnerd_small/articles.parquet')
history.head(2)

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"


In [66]:
topics = articles['topics'].explode().unique().drop_nans().drop_nulls().sort().to_frame().with_row_index()
category = articles['category'].unique().drop_nans().drop_nulls().sort().to_frame().with_row_index(offset=1)
subcategory = articles['subcategory'].explode().unique().drop_nans().drop_nulls().sort().to_frame().with_row_index()
mask = 0

articles = articles.select(['article_id', 'category', 'subcategory', 'premium', 'topics'])\
    .with_columns(
        pl.col('topics').fill_null(pl.lit([])),
        pl.col('subcategory').fill_null(pl.lit([]))
    )\
    .with_columns(
        pl.col('topics').list.eval(pl.element().replace(topics['topics'], topics['index'], default=None)).list.drop_nulls(),
        pl.col('category').replace(category['category'], category['index'], default=None).fill_null(mask),
        pl.col('subcategory').list.eval(pl.element().replace(subcategory['subcategory'], subcategory['index'], default=None)).list.drop_nulls(),
        pl.col('premium').cast(pl.Int8)
)
articles.head(2)

article_id,category,subcategory,premium,topics
i32,i64,list[u32],i8,list[u32]
3001353,5,[],0,"[25, 47]"
3003065,7,"[87, 88]",0,"[69, 13, 77]"


In [67]:
dummies_topics = articles.select('article_id', 'topics').explode('topics').drop_nulls().to_dummies(columns=['topics'])\
    .group_by('article_id').agg(pl.all().sum())
dummies_subcategories = articles.select('article_id', 'subcategory').explode('subcategory').drop_nulls().to_dummies(columns=['subcategory'])

dummies_subcategories

article_id,subcategory_0,subcategory_1,subcategory_10,subcategory_100,subcategory_101,subcategory_102,subcategory_103,subcategory_104,subcategory_105,subcategory_106,subcategory_107,subcategory_108,subcategory_109,subcategory_11,subcategory_110,subcategory_111,subcategory_112,subcategory_113,subcategory_114,subcategory_115,subcategory_116,subcategory_117,subcategory_118,subcategory_119,subcategory_12,subcategory_120,subcategory_121,subcategory_122,subcategory_123,subcategory_124,subcategory_125,subcategory_126,subcategory_127,subcategory_128,subcategory_129,subcategory_13,…,subcategory_66,subcategory_67,subcategory_68,subcategory_69,subcategory_7,subcategory_70,subcategory_71,subcategory_72,subcategory_73,subcategory_74,subcategory_75,subcategory_76,subcategory_77,subcategory_78,subcategory_79,subcategory_8,subcategory_80,subcategory_81,subcategory_82,subcategory_83,subcategory_84,subcategory_85,subcategory_86,subcategory_87,subcategory_88,subcategory_89,subcategory_9,subcategory_90,subcategory_91,subcategory_92,subcategory_93,subcategory_94,subcategory_95,subcategory_96,subcategory_97,subcategory_98,subcategory_99
i32,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
3003065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3003065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3012771,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3012771,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3023463,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9803455,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9803492,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9803505,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [68]:
articles = articles.join(dummies_topics, on='article_id', how='left')\
    .join(dummies_subcategories, on='article_id', how='left')\
    .drop('topics', 'subcategory')
    
one_hot_cols = [col for col in articles.columns if col.startswith('topics_') or col.startswith('subcategory_')]
articles = articles.with_columns(
    pl.col(one_hot_cols).fill_null(0)
)
articles.head(2)

article_id,category,premium,topics_0,topics_1,topics_10,topics_11,topics_12,topics_13,topics_14,topics_15,topics_16,topics_17,topics_18,topics_19,topics_2,topics_20,topics_21,topics_22,topics_23,topics_24,topics_25,topics_26,topics_27,topics_28,topics_29,topics_3,topics_30,topics_31,topics_32,topics_33,topics_34,topics_35,topics_36,topics_37,topics_38,topics_39,…,subcategory_66,subcategory_67,subcategory_68,subcategory_69,subcategory_7,subcategory_70,subcategory_71,subcategory_72,subcategory_73,subcategory_74,subcategory_75,subcategory_76,subcategory_77,subcategory_78,subcategory_79,subcategory_8,subcategory_80,subcategory_81,subcategory_82,subcategory_83,subcategory_84,subcategory_85,subcategory_86,subcategory_87,subcategory_88,subcategory_89,subcategory_9,subcategory_90,subcategory_91,subcategory_92,subcategory_93,subcategory_94,subcategory_95,subcategory_96,subcategory_97,subcategory_98,subcategory_99
i32,i64,i8,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16
3001353,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3003065,7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
df = pl.concat([
    slice.explode(pl.all().exclude('user_id'))\
        .with_columns(
            pl.col('scroll_percentage_fixed').fill_null(0.),
            pl.col('read_time_fixed').fill_null(0.),
        )\
        .with_columns(
            (pl.col('impression_time_fixed').dt.hour() // 4).alias('hour_group'),
            pl.col('impression_time_fixed').dt.weekday().alias('weekday'),
        ).drop('impression_time_fixed')\
        .rename({'scroll_percentage_fixed': 'scroll_percentage', 'read_time_fixed': 'read_time'})
        .join(articles, left_on='article_id_fixed', right_on='article_id', how='left').drop('article_id_fixed')\
        .group_by('user_id').agg(pl.all())
    for slice in history.iter_slices(10000)
])

df

user_id,scroll_percentage,read_time,hour_group,weekday,category,premium,topics_0,topics_1,topics_10,topics_11,topics_12,topics_13,topics_14,topics_15,topics_16,topics_17,topics_18,topics_19,topics_2,topics_20,topics_21,topics_22,topics_23,topics_24,topics_25,topics_26,topics_27,topics_28,topics_29,topics_3,topics_30,topics_31,topics_32,topics_33,topics_34,topics_35,…,subcategory_66,subcategory_67,subcategory_68,subcategory_69,subcategory_7,subcategory_70,subcategory_71,subcategory_72,subcategory_73,subcategory_74,subcategory_75,subcategory_76,subcategory_77,subcategory_78,subcategory_79,subcategory_8,subcategory_80,subcategory_81,subcategory_82,subcategory_83,subcategory_84,subcategory_85,subcategory_86,subcategory_87,subcategory_88,subcategory_89,subcategory_9,subcategory_90,subcategory_91,subcategory_92,subcategory_93,subcategory_94,subcategory_95,subcategory_96,subcategory_97,subcategory_98,subcategory_99
u32,list[f32],list[f32],list[i8],list[i8],list[i64],list[i8],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],…,list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16]
1711118,"[0.0, 36.0, … 50.0]","[11.0, 15.0, … 30.0]","[2, 2, … 1]","[4, 4, … 4]","[4, 4, … 4]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[1, 1, … 1]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[1, 0, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]",…,"[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]"
2291058,"[94.0, 69.0, … 100.0]","[83.0, 14.0, … 142.0]","[2, 2, … 1]","[4, 4, … 4]","[4, 6, … 6]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[1, 0, … 0]","[0, 0, … 0]","[1, 0, … 0]","[0, 1, … 1]","[0, 0, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]",…,"[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 1]","[1, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]"
1488980,"[100.0, 22.0, … 45.0]","[51.0, 2.0, … 0.0]","[2, 2, … 5]","[4, 4, … 3]","[7, 6, … 4]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[1, 0, … 1]","[0, 0, … 0]","[0, 0, … 0]","[1, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, …

In [70]:
cols = df.columns
topics_cols = sorted([col for col in cols if col.startswith('topics_')], key=lambda x: int(x.split('_')[-1]))
subcategory_cols = sorted([col for col in cols if col.startswith('subcategory_')], key=lambda x: int(x.split('_')[-1]))
all_others = set(cols) - set(topics_cols) - set(subcategory_cols) - {'user_id'}
cols = ['user_id'] + list(all_others) + topics_cols + subcategory_cols
df = df.select(cols)
df.head(1)

user_id,weekday,premium,scroll_percentage,hour_group,read_time,category,topics_0,topics_1,topics_2,topics_3,topics_4,topics_5,topics_6,topics_7,topics_8,topics_9,topics_10,topics_11,topics_12,topics_13,topics_14,topics_15,topics_16,topics_17,topics_18,topics_19,topics_20,topics_21,topics_22,topics_23,topics_24,topics_25,topics_26,topics_27,topics_28,topics_29,…,subcategory_137,subcategory_138,subcategory_139,subcategory_140,subcategory_141,subcategory_142,subcategory_143,subcategory_144,subcategory_145,subcategory_146,subcategory_147,subcategory_148,subcategory_149,subcategory_150,subcategory_151,subcategory_152,subcategory_153,subcategory_154,subcategory_155,subcategory_156,subcategory_157,subcategory_158,subcategory_159,subcategory_160,subcategory_161,subcategory_162,subcategory_163,subcategory_164,subcategory_165,subcategory_166,subcategory_167,subcategory_168,subcategory_169,subcategory_170,subcategory_171,subcategory_172,subcategory_173
u32,list[i8],list[i8],list[f32],list[i8],list[f32],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],…,list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16],list[i16]
1711118,"[4, 4, … 4]","[0, 0, … 0]","[0.0, 36.0, … 50.0]","[2, 2, … 1]","[11.0, 15.0, … 30.0]","[4, 4, … 4]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[1, 1, … 1]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[1, 0, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]",…,"[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]"


In [116]:

from tqdm import tqdm


def build_sequences(df: pl.DataFrame, w: int, stride: int):
    
    all_features = df.drop('user_id').columns
    singular_cols = ['topics', 'subcategory', 'category', 'weekday', 'hour_group']
    name_idx_dict = {key: [i for i, col in enumerate(all_features) if col.startswith(key)] for key in singular_cols}
    numerical_cols = ['scroll_percentage', 'read_time', 'premium']
    name_idx_dict['numerical'] = [i for i, col in enumerate(all_features) if col in numerical_cols]
        
    res = {key: ([], []) for key in name_idx_dict.keys()}

    for user_df in tqdm(df.partition_by('user_id')):
        x = user_df.drop('user_id').to_numpy()[0]
        x = np.array([np.array(x_i) for x_i in x])
                
        i = 0
        if i + w >= x.shape[1]:
            pad_width = w - x[:, :-1].shape[1]
            pad_m = np.zeros((x.shape[0], pad_width))
            padded_x = np.concatenate((pad_m, x[:, :-1]), axis=1)
            y_i = x[:, -1]
            
            for key, idx in name_idx_dict.items():
                res[key][0].append(padded_x[idx, :].T)
                res[key][1].append(y_i[idx].T)
            
        else:
            while i + w < x.shape[1]:
                y_i = x[:, i+w]
                x_i = x[:, i:i+w]
                
                for key, idx in name_idx_dict.items():
                    res[key][0].append(x_i[idx, :].T)
                    res[key][1].append(y_i[idx].T)
                
                i+=stride
            #TODO: add padding for the last sequence, if we want to keep it

    for key in res.keys():
        res[key] = (np.array(res[key][0]), np.array(res[key][1]))
    
    return res

In [120]:
res = build_sequences(df[:10], w=10, stride=5)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 54.12it/s]
